Data File ===> DataFrame, Numpy (전처리) ===> Tensor ===> Dataset(피처 + 타겟) ===> DataLoader 생성

### Dataset & DataLoader 살펴보기 <hr>
- Pytorch에서 배치크기만 데이터를 조절하기 위한 메커니즘
- Dataset = 사용 데이터를 기반으로 사용자정의 클래스 작성
- DataLoader : Dataset에서 지정된 batch size 만큼 피처와 타겟을 추출하여 전달

[1] 모듈 로딩 및 데이터 준비

In [67]:
# 모듈 로딩
import torch
import torch.nn as nn
import torch.optim as optim 
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd

In [68]:
# 데이터 준비
x_data = torch.IntTensor([[10, 20, 30], [20, 30, 40], [30, 40, 50], [40, 50, 60], [50, 60 ,70]])
y_data = torch.IntTensor([[20], [30], [40], [50], [60]])
print(f'x_data => {x_data.shape}, {x_data.ndim}')
print(f'y_data => {y_data.shape}, {y_data.ndim}')

x_data => torch.Size([5, 3]), 2
y_data => torch.Size([5, 1]), 2


[2] 데이터셋 생성

[2-1] TensorDataset 활용 : Dataset의 sub_class

In [69]:
# TensorDataset 클래스 로딩
from torch.utils.data import TensorDataset

In [70]:
dataset = TensorDataset(x_data, y_data)
dataset

In [71]:
dataset.tensors

(tensor([[10, 20, 30],
         [20, 30, 40],
         [30, 40, 50],
         [40, 50, 60],
         [50, 60, 70]], dtype=torch.int32),
 tensor([[20],
         [30],
         [40],
         [50],
         [60]], dtype=torch.int32))

In [72]:
# __getitem__() 메서드 호출
dataset[0]

(tensor([10, 20, 30], dtype=torch.int32), tensor([20], dtype=torch.int32))

In [73]:
len(dataset)

5

In [87]:
filename = '../DATA/Iris.csv'
irisDF = pd.read_csv(filename)

In [88]:
irisNP = np.loadtxt(filename, delimiter = ',', usecols = [0, 1, 2, 3], skiprows = 1)
irisNP[:2]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2]])

In [89]:
## 데이터의 타입 체크
type(irisDF), type(irisNP), irisDF.__class__.__name__, irisNP.__class__.__name__

(pandas.core.frame.DataFrame, numpy.ndarray, 'DataFrame', 'ndarray')

In [90]:
isinstance(irisDF, pd.DataFrame), isinstance(irisNP, pd.DataFrame), isinstance(irisNP, np.ndarray)

(True, False, True)

In [95]:
# 사용자 정의 DataSet 클래스
class DLDataset(Dataset):

    # 초기화 콜백 함수(callback function)
    def __init__(self, x_data, y_data):
        super().__init__()
        x_data = x_data.values if isinstance(x_data, pd.DataFrame) else x_data
        y_data = y_data.values if isinstance(y_data, pd.DataFrame) else y_data
        self.feature = torch.tensor(x_data)
        self.target = torch.tensor(y_data)

        # ndarray => tensor
        self.feature = torch.FloatTensor(x_data)
        self.target = torch.FloatTensor(y_data)
    
    # 데이터셋의 개수 체크 콜백 함수(callback function)
    def __len__(self):
        return self.target.shape[0]
    
    # 특정 인덱스 데이터 + 라벨 반환 콜백 함수(callback function)
    def __getitem__(self, index):
        return self.feature[index], self.target[index]

In [96]:
# 피처와 라벨 분리
featureDF = irisDF[irisDF.columns[:-1]]
targetSR = irisDF[irisDF.columns[-1]]

print(f'featureDF => {featureDF.shape}, {featureDF.ndim}D')
print(f'targerSR => {targetSR.shape}, {targetSR.ndim}D')

featureDF => (150, 4), 2D
targerSR => (150,), 1D


In [97]:
# 타겟이 object이므로 정수로 변환
from sklearn.preprocessing import LabelEncoder

targetNP = LabelEncoder().fit_transform(targetSR)
targetNP = targetNP.reshape(-1, 1)
print(targetNP.shape, targetNP.ndim)

(150, 1) 2


In [98]:
# 데이터셋 생성 => DF, NP
my_dataset = DLDataset(featureDF, targetNP)
my_dataset[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0.]))

In [99]:
# 데이터셋 생성 => NP, NP
my_dataset2 = DLDataset(irisNP, targetNP)
my_dataset2[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0.]))

[2-3] 학습용, 검증용, 테스트용 Dataset

In [100]:
# 파이토치
from torch.utils.data import random_split

# 학습용, 검증용, 테스트 데이터 비율
seed = torch.Generator().manual_seed(42)

trainDS, validDS, testDS = random_split(my_dataset2, [0.7, 0.1, 0.2], generator = seed)

print(f'trainDS =>{len(trainDS)}개, validDS => {len(validDS)}개, testDS => {len(testDS)}개')

print(f'Subset 속성 =>\nindices : {trainDS.indices} \nindices : {trainDS.dataset}')
print(f'Subset 속성 =>\nindices : {validDS.indices} \nindices : {validDS.dataset}')
print(f'Subset 속성 =>\nindices : {testDS.indices} \nindices : {testDS.dataset}')

trainDS =>105개, validDS => 15개, testDS => 30개
Subset 속성 =>
indices : [42, 95, 30, 64, 52, 35, 130, 40, 82, 17, 108, 94, 68, 97, 117, 127, 41, 44, 57, 140, 149, 32, 23, 102, 16, 113, 71, 18, 67, 66, 0, 25, 101, 112, 91, 3, 59, 116, 86, 84, 106, 142, 43, 39, 26, 98, 93, 20, 87, 19, 120, 114, 7, 63, 76, 89, 36, 45, 37, 56, 58, 122, 51, 145, 24, 21, 105, 62, 15, 11, 48, 133, 88, 50, 6, 134, 111, 8, 49, 75, 69, 124, 4, 147, 80, 100, 99, 141, 47, 107, 13, 109, 129, 28, 38, 53, 121, 5, 55, 31, 73, 74, 54, 29, 12] 
indices : <__main__.DLDataset object at 0x0000019FB08CC1C0>
Subset 속성 =>
indices : [22, 104, 81, 1, 103, 125, 85, 2, 96, 128, 27, 118, 77, 110, 146] 
indices : <__main__.DLDataset object at 0x0000019FB08CC1C0>
Subset 속성 =>
indices : [72, 139, 131, 60, 65, 92, 135, 83, 14, 34, 137, 10, 119, 9, 148, 79, 78, 70, 144, 143, 123, 115, 61, 132, 90, 46, 126, 136, 33, 138] 
indices : <__main__.DLDataset object at 0x0000019FB08CC1C0>


[3] DataLoader 생성 : 학습용, 검증용, 테스트용

In [101]:
# DataLoader 생성
# drop_last 매개변수 : 배치 사이즈로 데이터셋 분리 후 남는 데이터 처리 방법 설정 [기본 : False]
batch = 10
trainDL = DataLoader(trainDS, batch_size = batch, drop_last = True)
validDL = DataLoader(validDS, batch_size = batch)
testDL = DataLoader(testDS, batch_size = batch)

len(trainDL), len(validDL), len(testDL)

(10, 2, 3)

In [102]:
# Epoch당 반복 단위
print(f'batch_size = {batch}')
print(f'trainDS => {len(trainDS)}개, validDS => {len(validDS)}개, testDS => {len(testDS)}개')
print(f'trainDL => {len(trainDL)}개, validDL => {len(validDL)}개, testDL => {len(testDL)}개')

batch_size = 10
trainDS => 105개, validDS => 15개, testDS => 30개
trainDL => 10개, validDL => 2개, testDL => 3개


In [103]:
batch = 10
trainDL = DataLoader(trainDS, batch_size = batch, drop_last = True)
validDL = DataLoader(validDS, batch_size = batch)
testDL = DataLoader(testDS, batch_size = batch)

len(trainDL), len(validDL), len(testDL)

(10, 2, 3)

In [104]:
# DataLoader 속성
for _, (feature, target) in enumerate(validDL):
    print(f'[{_}] feature {feature}')

[0] feature tensor([[4.6000, 3.6000, 1.0000, 0.2000],
        [6.5000, 3.0000, 5.8000, 2.2000],
        [5.5000, 2.4000, 3.7000, 1.0000],
        [4.9000, 3.0000, 1.4000, 0.2000],
        [6.3000, 2.9000, 5.6000, 1.8000],
        [7.2000, 3.2000, 6.0000, 1.8000],
        [6.0000, 3.4000, 4.5000, 1.6000],
        [4.7000, 3.2000, 1.3000, 0.2000],
        [5.7000, 2.9000, 4.2000, 1.3000],
        [6.4000, 2.8000, 5.6000, 2.1000]])
[1] feature tensor([[5.2000, 3.5000, 1.5000, 0.2000],
        [7.7000, 2.6000, 6.9000, 2.3000],
        [6.7000, 3.0000, 5.0000, 1.7000],
        [6.5000, 3.2000, 5.1000, 2.0000],
        [6.3000, 2.5000, 5.0000, 1.9000]])
